# Streaming multispeaker ASR and diarization tutorial

Set your NeMo path

In [6]:
import sys

if socket.gethostname() == "aiapps-06052021":
    sys.path.insert(0,'/home/taejinp/projects/_streaming_mulspk_asr/NeMo')
else:
    sys.path.insert(0,'/your/path/to/NeMo/')
    
import nemo
print("Using Nemo PATH:", nemo.__path__[0])

# !pip install gradio==2.9.0

Using Nemo PATH: /home/taejinp/projects/_streaming_mulspk_asr/NeMo/nemo


Set your NeMo path

In [7]:
from nemo.collections.asr.parts.utils.speaker_utils import audio_rttm_map
from nemo.core.config import hydra_runner
import gradio as gr
from scipy.io import wavfile
import numpy as np
import hydra
import os
import torch
from nemo.collections.asr.models import OnlineDiarizer
from nemo.collections.asr.parts.utils.diarization_utils import ASR_DIAR_ONLINE

Read yaml file for online diarization. You have to specifty the following items:
    
- input manifest file (If  simulation)
- VAD model path
- Speaker embedding extractor model path
- Diarization Decoder model path (Coming soon)
- Punctuation model path (automatically download from NGC)
- Language model path (Coming soon)

Download nemo models and specify the path to config struct.

In [3]:
hydra.initialize(config_path="conf")
cfg = hydra.compose(config_name="/online_diarization_with_asr.yaml")
import socket
print(socket.gethostname())
cfg.diarizer.out_dir = "./streaming_diar_output"
os.makedirs(cfg.diarizer.out_dir, exist_ok=True)

if socket.gethostname() == "aiapps-06052021":
    cfg.diarizer.manifest_filepath = "/home/taejinp/projects/data/diar_manifest_input/ch109.json"
    cfg.diarizer.vad.model_path = "/home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo"
    cfg.diarizer.speaker_embeddings.model_path = "/home/taejinp/Downloads/titanet-m.nemo"
    cfg.diarizer.asr.model_path = "/home/taejinp/gdrive/model/ASR_models/Conformer-CTC-BPE_large_Riva_ASR_set_3.0_ep60.nemo"
    cfg.diarizer.asr.parameters.punctuation_model_path = "punctuation_en_distilbert"
else:
    # Please download the following models and run the code. 

    # Download CH109 dataset at: https://drive.google.com/drive/folders/1ksq10H-NZbKRfMjEP_WWyBF_G0iAJt6b?usp=sharing
    cfg.diarizer.manifest_filepath = "/your/path/to/ch109.json"

    # Download streaming VAD model at: https://drive.google.com/file/d/1ab42CaYeTkuJSMsMsMLbSS9m5e1isJzx/view?usp=sharing
    cfg.diarizer.vad.model_path = "/your/path/to/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo"

    # Download titanet-m model at: https://drive.google.com/file/d/1xAgjm0udKogPrlQF6cdHLobEKHLY9azA/view?usp=sharing
    cfg.diarizer.speaker_embeddings.model_path = "/your/path/to/titanet-m.nemo"

    # Download Conformer-CTC ASR model at: https://drive.google.com/file/d/1Xg075IbiwL0szI4_a8gYmCPaG1UsgR6E/view?usp=sharing
    cfg.diarizer.asr.model_path = "/your/path/to/Conformer-CTC-BPE_large_Riva_ASR_set_3.0_ep60.nemo"

    cfg.diarizer.asr.parameters.punctuation_model_path = "punctuation_en_distilbert"

aiapps-06052021


Initialize ASR_DIAR_ONLINE and OnlineDiarizer Class.

In [8]:
params = {}
params['use_cuda'] = True
AUDIO_RTTM_MAP = audio_rttm_map(cfg.diarizer.manifest_filepath)

diar = OnlineDiarizer(cfg)

diar.uniq_id = "en_0638"
diar.single_audio_file_path = AUDIO_RTTM_MAP[diar.uniq_id]['audio_filepath']
diar.rttm_file_path = AUDIO_RTTM_MAP[diar.uniq_id]['rttm_filepath']
# diar.rttm_file_path = None # DER calculation slows down online diarization speed
diar._init_segment_variables()

asr_diar = ASR_DIAR_ONLINE(diar, cfg=cfg.diarizer)
diar.device = asr_diar.device
asr_diar.reset()

simulation = True
# simulation = False # Run Gradio server with your microphone.

[NeMo I 2022-06-22 21:43:33 speaker_utils:81] Number of files to diarize: 109


[NeMo W 2022-06-22 21:43:33 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2022-06-22 21:43:33 features:200] PADDING: 16


[NeMo W 2022-06-22 21:43:33 nemo_logging:349] /home/taejinp/anaconda3/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (TopKClassificationAccuracy). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-06-22 21:43:33 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from /home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo.
[NeMo I 2022-06-22 21:43:33 clustering_diarizer:122] VAD model loaded locally from /home/taejinp/gdrive/model/VAD_models/mVAD_lin_marblenet-3x2x64-4N-256bs-50e-0.01lr-0.001wd.nemo


...

[NeMo I 2022-06-22 21:43:43 features:200] PADDING: 0
[NeMo I 2022-06-22 21:43:44 save_restore_connector:243] Model EncDecCTCModelBPE was successfully restored from /home/taejinp/gdrive/model/ASR_models/Conformer-CTC-BPE_large_Riva_ASR_set_3.0_ep60.nemo.


[NeMo W 2022-06-22 21:43:44 decoder_timestamps_utils:69] `ctc_decode` was set to True. Note that this is ignored.


[NeMo I 2022-06-22 21:43:45 features:200] PADDING: 0
[NeMo I 2022-06-22 21:43:45 features:200] PADDING: 0
[NeMo I 2022-06-22 21:43:45 features:200] PADDING: 0
[NeMo I 2022-06-22 21:43:45 cloud:56] Found existing object /home/taejinp/.cache/torch/NeMo/NeMo_1.11.0rc0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.
[NeMo I 2022-06-22 21:43:45 cloud:62] Re-using file from: /home/taejinp/.cache/torch/NeMo/NeMo_1.11.0rc0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2022-06-22 21:43:45 common:789] Instantiating model from pre-trained checkpoint
[NeMo I 2022-06-22 21:43:46 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmp69aap1_7/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-06-22 21:43:48 modelPT:215] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-06-22 21:43:48 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shuffle: true
    drop_last: false
    pin_memory: true
    num_workers: 8
    persistent_workers: true
    ds_item: punct_dataset_complete
    
[NeMo W 2022-06-22 21:43:48 m

[NeMo I 2022-06-22 21:43:50 save_restore_connector:243] Model PunctuationCapitalizationModel was successfully restored from /home/taejinp/.cache/torch/NeMo/NeMo_1.11.0rc0/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.


Let's run simulated audio stream to check if streaming system is working properly. After you initiate the following function and while the function is running, you can check the transcription is being generated in realtime.  The path is ./streaming_diar_output/print_script.sh, and this can be viewed using "streaming_diarization_viewer.ipynb"


In [5]:

if simulation:
    samplerate, sdata = wavfile.read(diar.single_audio_file_path)
    for index in range(int(np.floor(sdata.shape[0]/asr_diar.n_frame_len))):
        asr_diar.buffer_counter = index
        sample_audio = sdata[asr_diar.CHUNK_SIZE*(asr_diar.buffer_counter):asr_diar.CHUNK_SIZE*(asr_diar.buffer_counter+1)]
        asr_diar.callback_sim(sample_audio)
else:
    isTorch = torch.cuda.is_available()
    asr_diar.rttm_file_path = None
    iface = gr.Interface(
        fn=asr_diar.audio_queue_launcher,
        inputs=[
            gr.inputs.Audio(source="microphone", type='filepath'), 
            "state",
        ],
        outputs=[
            "textbox",
            "state",
        ],
        layout="horizontal",
        theme="huggingface",
        title=f"NeMo Streaming Conformer CTC Large - English, CUDA:{isTorch}",
        description="Demo for English speech recognition using Conformer Transducers",
        allow_flagging='never',
        live=True,
    )
    iface.launch(share=False)


[NeMo I 2022-06-22 21:40:57 diarization_utils:562] Creating results for Session: en_0638 n_spk: 1 
[NeMo I 2022-06-22 21:40:57 diarization_utils:1636] Total ASR and Diarization ETA: 0.093 comp ETA 0.093
[NeMo I 2022-06-22 21:40:57 punctuation_capitalization_model:1091] Using batch size 1 for inference
[NeMo I 2022-06-22 21:40:57 punctuation_capitalization_infer_dataset:91] Max length: 5
[NeMo I 2022-06-22 21:40:57 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-06-22 21:40:57 data_preprocessing:406] Min: 3 |                  Max: 3 |                  Mean: 3.0 |                  Median: 3.0
[NeMo I 2022-06-22 21:40:57 data_preprocessing:412] 75 percentile: 3.00
[NeMo I 2022-06-22 21:40:57 data_preprocessing:413] 99 percentile: 3.00


100%|██████████| 1/1 [00:00<00:00, 170.38batch/s]

[NeMo I 2022-06-22 21:40:57 diarization_utils:562] Creating results for Session: en_0638 n_spk: 1 
[NeMo I 2022-06-22 21:40:57 diarization_utils:1636] Total ASR and Diarization ETA: 0.054 comp ETA 0.054
[NeMo I 2022-06-22 21:40:57 punctuation_capitalization_model:1091] Using batch size 1 for inference
[NeMo I 2022-06-22 21:40:57 punctuation_capitalization_infer_dataset:91] Max length: 7
[NeMo I 2022-06-22 21:40:57 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-06-22 21:40:57 data_preprocessing:406] Min: 5 |                  Max: 5 |                  Mean: 5.0 |                  Median: 5.0
[NeMo I 2022-06-22 21:40:57 data_preprocessing:412] 75 percentile: 5.00
[NeMo I 2022-06-22 21:40:57 data_preprocessing:413] 99 percentile: 5.00



100%|██████████| 1/1 [00:00<00:00, 180.10batch/s]

[NeMo I 2022-06-22 21:40:58 diarization_utils:562] Creating results for Session: en_0638 n_spk: 1 
[NeMo I 2022-06-22 21:40:58 diarization_utils:1636] Total ASR and Diarization ETA: 0.052 comp ETA 0.052
[NeMo I 2022-06-22 21:40:58 punctuation_capitalization_model:1091] Using batch size 1 for inference
[NeMo I 2022-06-22 21:40:58 punctuation_capitalization_infer_dataset:91] Max length: 11
[NeMo I 2022-06-22 21:40:58 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-06-22 21:40:58 data_preprocessing:406] Min: 9 |                  Max: 9 |                  Mean: 9.0 |                  Median: 9.0
[NeMo I 2022-06-22 21:40:58 data_preprocessing:412] 75 percentile: 9.00
[NeMo I 2022-06-22 21:40:58 data_preprocessing:413] 99 percentile: 9.00


...

100%|██████████| 1/1 [00:00<00:00, 83.33batch/s]


[NeMo I 2022-06-22 21:41:52 diarization_utils:1636] Total ASR and Diarization ETA: 1.034 comp ETA 1.034


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/taejinp/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-51e21b2b3d9b>", line 6, in <module>
    asr_diar.callback_sim(sample_audio)
  File "/home/taejinp/projects/_streaming_mulspk_asr/NeMo/nemo/collections/asr/parts/utils/diarization_utils.py", line 1613, in callback_sim
    words, timestamps, diar_hyp = self.transcribe(sample_audio)
  File "/home/taejinp/anaconda3/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/home/taejinp/projects/_streaming_mulspk_asr/NeMo/nemo/collections/asr/parts/utils/diarization_utils.py", line 1666, in transcribe
    self.diar_hyp = self.diar.online_diarization(self, vad_ts)
  File "/home/taejinp/projects/_streaming_mulspk_asr/NeMo/nemo/collections/asr/parts/utils/diarization_utils.py", line 1165, in timed

TypeError: object of type 'NoneType' has no len()

Now, go to streaming_diarization_viewer.ipynb and check the realtime output.